In [1]:
#import necessary libraries
import os
import requests
import pipecat
import re
from mistralai.client import MistralClient

In [2]:
# Set API key for Mistral AI
os.environ["MISTRAL_API_KEY"] = "SECRETMISTRALKEY"

# Set OpenWeather API key
os.environ["OPENWEATHER_API_KEY"] = "SUPERSECRETOPENWEATHERKEY"

In [3]:
# Verify API keys are set correctly
print("Mistral API Key:", os.getenv("MISTRAL_API_KEY"))
print("OpenWeather API Key:", os.getenv("OPENWEATHER_API_KEY"))

Mistral API Key: SECRETMISTRALKEY
OpenWeather API Key: SUPERSECRETOPENWEATHERKEY


In [4]:
client = MistralClient(api_key=os.environ["MISTRAL_API_KEY"])

In [5]:
#function to chat with Mistral
def chat_with_mistral(prompt):
    response = client.chat(
        model="mistral-tiny", 
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

print(chat_with_mistral("Hello, how are you?"))

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with any questions you have. How can I assist you today?


In [6]:
def chat_with_mistral(prompt):
    # Check if the message contains "weather"
    if "weather" in prompt.lower():
        return "It looks like you're asking about the weather! Soon, we'll fetch real data."

    # Otherwise, send the message to Mistral AI
    response = client.chat(
        model="mistral-small",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

# Test with a weather-related question
print(chat_with_mistral("What's the weather like in Berlin?"))
print(chat_with_mistral("Tell me a joke."))

It looks like you're asking about the weather! Soon, we'll fetch real data.
Why don't scientists trust atoms? Because they make up everything!


In [7]:
# Function to fetch weather data from OpenWeather API
def get_weather(city):
    openweather_api_key = os.getenv("OPENWEATHER_API_KEY")
    if not openweather_api_key:
        return "API key is missing. Set the OPENWEATHER_API_KEY."

    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_api_key}&units=metric"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        weather = data["weather"][0]["description"]
        temp = data["main"]["temp"]
        return f"The weather in {city} is {weather} with a temperature of {temp}°C."
    else:
        return f"Error fetching weather data: {response.json().get('message', 'Unknown error')}"

In [8]:
# Set OpenWeather API key 
openweather_api_key = os.getenv("OPENWEATHER_API_KEY")

# Test again
print(get_weather("Berlin"))

The weather in Berlin is overcast clouds with a temperature of -0.86°C.


In [9]:
def chat_with_mistral(prompt):
    # Check if the message contains "weather"
    if "weather" in prompt.lower():
        # Try extracting a city name using regex
        city_match = re.search(r"in ([A-Za-z\s]+)", prompt)
        if city_match:
            city = city_match.group(1).strip()
        else:
            return "I need a city name to check the weather!"

        # Call the real weather API function
        return get_weather(city)

    # Otherwise, send the message to Mistral AI
    response = client.chat(
        model="mistral-small",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

# Test with a real weather request
print(chat_with_mistral("What's the weather like in Berlin?"))
print(chat_with_mistral("Tell me a joke."))

The weather in Berlin is overcast clouds with a temperature of -0.86°C.
Of course, I'd be happy to share a joke with you! Here it is:

Why don't scientists trust atoms?

Because they make up everything!

I hope that brought a smile to your face. Would you like to hear another joke, or is there something else you'd like to talk about?


In [10]:
def extract_city(prompt):
    """Extracts a city name from a user's prompt and ensures it's valid."""
    # Check for 'in CITY'
    city_match = re.search(r"in ([A-Za-z\s]+)", prompt)
    if city_match:
        city = city_match.group(1).strip()
        if len(city) > 2 and city.lower() not in ["weather", "temperature", "like"]:
            return city

    words = prompt.split()
    last_word = words[-1].strip("?.!,").capitalize()

    if len(last_word) > 2 and last_word.lower() not in ["weather", "temperature", "like"]:
        return last_word

    return None  # No valid city detected

# chatbot function
def chat_with_mistral(prompt):
    # Check if the message contains "weather"
    if "weather" in prompt.lower():
        city = extract_city(prompt)

        # If no valid city was found, ask the user
        if not city:
            city = input("Which city's weather do you want to know? ").strip()
            return get_weather(city)

        # Call the weather API function
        return get_weather(city)

    # Otherwise, send the message to Mistral AI
    response = client.chat(
        model="mistral-small",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

# Test chatbot interactions
print(chat_with_mistral("What's the weather like?"))  
print(chat_with_mistral("Weather Tokyo"))  
print(chat_with_mistral("Tell me a joke."))  

Which city's weather do you want to know?  frankfurt


The weather in frankfurt is overcast clouds with a temperature of 0.84°C.
The weather in Tokyo is overcast clouds with a temperature of 8.75°C.
Of course, I'd be happy to share a joke with you! Here it is:

Why don't scientists trust atoms?

Because they make up everything!

I hope that brought a smile to your face. Would you like to hear another joke, or is there something else you'd like to talk about?
